<a href="https://colab.research.google.com/github/radonys/webscrapers/blob/master/Facebook_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facebook Scraper

## Install Modules

In [0]:
!pip install facebook-scraper

## Import Modules

In [0]:
from facebook_scraper import get_posts

import pandas as pd
import os
import requests
import shutil
import sys
import re

## Variable Declaration

In [0]:
facebook_profiles = ['AshleyHomeStore', 'bonobos', 'CocaColaUnitedStates', 'IKEAUSA', 'lazboy', 'potterybarn', 'raymourflanigan', 'SimplyGum', 'starfurnituretx', 'WilliamsSonoma', 'walmart']

## Helper Functions

### Retrieve Posts and Save Data

References:

1) Facebook Scraper ([GitHub](https://github.com/kevinzg/facebook-scraper))

In [0]:
def scrape_facebook_posts(profile):
  
  posts = []
  prev_post_count = len(posts)
  tries = 10
  
  while tries>=0:

    for post in get_posts(profile, pages=sys.maxsize, sleep=0, timeout=10):
      if post['image']:
        posts.append(post)
    
    posts = list({post['post_id']:post for post in posts}.values())
    
    if len(posts) <= prev_post_count:
      tries -= 1
      
    prev_post_count = len(posts)
    #print(tries, prev_post_count)
  
  data = pd.DataFrame.from_dict(posts, orient='columns')
  data.to_csv(profile+'_data.csv')
  save_images(posts, profile+'_images')

### Save Images

In [0]:
def save_images(posts, save_name):
  
  images_path = save_name+'/'
  
  if not os.path.isdir(images_path):
    os.mkdir(images_path)
  
  for post in posts:

    image = requests.get(post['image'])
    
    try:

      with open(images_path + post['post_id'] + ".jpg", 'wb') as file:
        file.write(image.content)
    
    except Exception as error:
      print('Caught this error: ' + repr(error))

## Scrape Facebook Data

In [0]:
for profile in facebook_profiles:
  
  scrape_facebook_posts(profile)
  shutil.make_archive(profile+'_images', 'zip', profile+'_images')